# In this case study lets implement Boosting regressors on boston dataset

Lets explore

* GBDT
* ADAboost
* XGboost
* LightGBM
* Cat Boost

# About Dataset

The Boston Housing Dataset

The Boston Housing Dataset is a derived from information collected by the U.S. Census Service concerning housing in the area of Boston MA. The following describes the dataset columns:

* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per 10,000 dollors
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in 1000 dollor's

## Import Section

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sks
from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,median_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
# https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn
from xgboost import XGBRegressor
# https://lightgbm.readthedocs.io/en/v3.3.2/pythonapi/lightgbm.LGBMRegressor.html
from lightgbm import LGBMRegressor
# https://catboost.ai/en/docs/concepts/python-reference_catboostregressor
# https://catboost.ai/en/docs/concepts/python-quickstart
!pip install catboost
from catboost import CatBoostRegressor, Pool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.5 MB/s 


Lets load the data

In [2]:
boston = load_boston()
print(boston.data.shape)

(506, 13)


In [3]:
print(boston.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [4]:
bos = pd.DataFrame(boston.data)
print(bos.head())

        0     1     2    3      4      5     6       7    8      9     10  \
0  0.00632  18.0  2.31  0.0  0.538  6.575  65.2  4.0900  1.0  296.0  15.3   
1  0.02731   0.0  7.07  0.0  0.469  6.421  78.9  4.9671  2.0  242.0  17.8   
2  0.02729   0.0  7.07  0.0  0.469  7.185  61.1  4.9671  2.0  242.0  17.8   
3  0.03237   0.0  2.18  0.0  0.458  6.998  45.8  6.0622  3.0  222.0  18.7   
4  0.06905   0.0  2.18  0.0  0.458  7.147  54.2  6.0622  3.0  222.0  18.7   

       11    12  
0  396.90  4.98  
1  396.90  9.14  
2  392.83  4.03  
3  394.63  2.94  
4  396.90  5.33  


In [5]:
bos['PRICE'] = boston.target

X = bos.drop('PRICE', axis = 1)
y = bos['PRICE']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
result_dataset = pd.DataFrame(data =np.zeros((6,9)) , index = ['r2_score','adjusted_r2_score','median_absolute_error','mean_squared_error','mean_absolute_percentage_error','mean_absolute_error'],columns = ['GBDT','ADABOOST_linear','ADABOOST_polynomial','ADABOOST_elasticnet','ADABOOST_svr','ADABOOST_dt','XGBOOST','LGBM','CATBOOST'])
result_dataset

,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
def metrics(y_true,y_pred,model = 'Knearest_neighbours'):
  r2_scores = r2_score(y_true,y_pred)
  p = 13 # number of features in the test dataset
  adjusted_r2_scores = 1- (1- r2_score(y_true,y_pred))*(len(y_true)-1)/(len(y_true)-p-1)
  median_absolute_errors = median_absolute_error(y_true,y_pred)
  mean_squared_errors = mean_squared_error(y_true,y_pred)
  mean_absolute_percentage_errors = mean_absolute_percentage_error(y_true,y_pred)
  mean_absolute_errors = mean_absolute_error(y_true,y_pred)
  result_dataset[model]['r2_score'] = r2_scores
  result_dataset[model]['adjusted_r2_score'] = adjusted_r2_scores
  result_dataset[model]['median_absolute_error'] = median_absolute_errors
  result_dataset[model]['mean_squared_error'] = mean_squared_errors
  result_dataset[model]['mean_absolute_percentage_error'] = mean_absolute_percentage_errors
  result_dataset[model]['mean_absolute_error'] = mean_absolute_errors
  print('r2_score for '+ model + " :" + str(r2_scores))
  print('adjusted_r2_score for '+ model + " :" + str(adjusted_r2_scores)) # Adj r2 = 1-(1-R2)*(n-1)/(n-p-1) Where n is the sample size and p is the number of independent variables.
  print('median_absolute_error for '+ model + " :" + str(median_absolute_errors))
  print('mean_squared_error for '+ model + " :" + str(mean_squared_errors))
  print('mean_absolute_percentage_error for '+ model + " :" + str(mean_absolute_percentage_errors))
  print('mean_absolute_error for '+ model + " :" + str(mean_absolute_errors))

# lets start with GBDT

In [10]:
gbdt_reg = GradientBoostingRegressor(random_state = 42)
param_grids = {'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],'learning_rate' : [0.1,0.3,0.6,0.9],'n_estimators':[80,90,100,110],'criterion':['friedman_mse','squared_error','mse'],'max_depth': [1,2,3,4,5]}
gbdt_reg = GridSearchCV(estimator = gbdt_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
gbdt_reg.fit(X_train,y_train)
gbdt_reg.best_estimator_

Fitting 5 folds for each of 960 candidates, totalling 4800 fits
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=80; total time=   0.1s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=80; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=80; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=80; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=80; total time=   0.0s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=90; total time=   0.1s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_estimators=90; total time=   0.1s
[CV] END criterion=friedman_mse, learning_rate=0.1, loss=squared_error, max_depth=1, n_es

GradientBoostingRegressor(learning_rate=0.3, loss='huber', n_estimators=90,
                          random_state=42)

GradientBoostingRegressor(learning_rate=0.3, loss='huber', n_estimators=90,
                          random_state=42)

In [45]:
gbdt_reg = GradientBoostingRegressor(learning_rate=0.3, loss='huber', n_estimators=90, random_state=42,max_depth=4,criterion = 'mse')
gbdt_reg.fit(X_train,y_train)
y_pred = gbdt_reg.predict(X_test)
metrics(y_test,y_pred,model = 'GBDT')
result_dataset

r2_score for GBDT :0.9077657732960193
adjusted_r2_score for GBDT :0.8941402625329312
median_absolute_error for GBDT :1.1929763370676945
mean_squared_error for GBDT :6.763884781069791
mean_absolute_percentage_error for GBDT :0.09658876790474871
mean_absolute_error for GBDT :1.8271276409986321


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.894140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.192976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,6.763885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.096589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,1.827128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# lets start with ADAboost with linear reg

In [18]:
basemodel = LinearRegression()
adalinear_reg = AdaBoostRegressor(base_estimator = basemodel,random_state = 42)
param_grids = {'learning_rate' : [0.01,0.1,1.0,3.0],'n_estimators':[5,10,20,30,35],'loss':['linear', 'square', 'exponential']}
adalinear_reg = GridSearchCV(estimator = adalinear_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
adalinear_reg.fit(X_train,y_train)
adalinear_reg.best_estimator_

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=20; total time=   0.0s
[CV] END ...learning_rate=0.01, loss=linear, n_

AdaBoostRegressor(base_estimator=LinearRegression(), learning_rate=0.01,
                  loss='exponential', n_estimators=35, random_state=42)

AdaBoostRegressor(base_estimator=LinearRegression(), learning_rate=0.01,
                  loss='exponential', n_estimators=35, random_state=42)

In [46]:
adalinear_reg = AdaBoostRegressor(base_estimator=LinearRegression(), learning_rate=0.01,loss='exponential', n_estimators=35, random_state=42)
adalinear_reg.fit(X_train,y_train)
y_pred = adalinear_reg.predict(X_test)
metrics(y_test,y_pred,model = 'ADABOOST_linear')
result_dataset

r2_score for ADABOOST_linear :0.6709879386928823
adjusted_r2_score for ADABOOST_linear :0.6223838841816036
median_absolute_error for ADABOOST_linear :2.366688484032972
mean_squared_error for ADABOOST_linear :24.12769916102705
mean_absolute_percentage_error for ADABOOST_linear :0.16801025779325596
mean_absolute_error for ADABOOST_linear :3.1762614551876025


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.0,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.894140,0.622384,0.0,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.192976,2.366688,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,6.763885,24.127699,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,1.827128,3.176261,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# lets start with ADAboost with polynomial linear reg

In [20]:
polynomial = PolynomialFeatures()
X_train_poly = polynomial.fit_transform(X_train)
X_test_poly =  polynomial.transform(X_test) 

In [21]:
print(X_train.shape)
print(X_train_poly.shape)
print(X_test.shape)
print(X_test_poly.shape)

(404, 13)
(404, 105)
(102, 13)
(102, 105)


In [22]:
basemodel = LinearRegression()
adalinear_reg = AdaBoostRegressor(base_estimator = basemodel,random_state = 42)
param_grids = {'learning_rate' : [0.01,0.1,1.0,3.0],'n_estimators':[5,10,20,30,35],'loss':['linear', 'square', 'exponential']}
adalinear_reg = GridSearchCV(estimator = adalinear_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
adalinear_reg.fit(X_train_poly,y_train)
adalinear_reg.best_estimator_

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.1s
[CV] END ....learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.1s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.1s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.1s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.1s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.1s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.1s
[CV] END ...learning_rate=0.01, loss=linear, n_estimators=20; total time=   0.2s
[CV] END ...learning_rate=0.01, loss=linear, n_

AdaBoostRegressor(base_estimator=LinearRegression(), learning_rate=3.0,
                  loss='exponential', n_estimators=35, random_state=42)

AdaBoostRegressor(base_estimator=LinearRegression(), learning_rate=3.0,
                  loss='exponential', n_estimators=35, random_state=42)

In [47]:
adapolylinear_reg = AdaBoostRegressor(base_estimator=LinearRegression(), learning_rate=3.0,loss='exponential', n_estimators=35, random_state=42)
adapolylinear_reg.fit(X_train_poly,y_train)
y_pred = adapolylinear_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'ADABOOST_polynomial')
result_dataset

r2_score for ADABOOST_polynomial :0.7195966145255193
adjusted_r2_score for ADABOOST_polynomial :0.67817338712588
median_absolute_error for ADABOOST_polynomial :2.1340241004054246
mean_squared_error for ADABOOST_polynomial :20.563041067806022
mean_absolute_percentage_error for ADABOOST_polynomial :0.1726325504700779
mean_absolute_error for ADABOOST_polynomial :3.070066624591683


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.0,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.894140,0.622384,0.678173,0.0,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.192976,2.366688,2.134024,0.0,0.0,0.0,0.0,0.0,0.0
mean_squared_error,6.763885,24.127699,20.563041,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.0,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,1.827128,3.176261,3.070067,0.0,0.0,0.0,0.0,0.0,0.0


# lets start with ADAboost with Elasticnet linear reg

In [24]:
basemodel = ElasticNet(random_state = 42)
adaelastic_reg = AdaBoostRegressor(base_estimator = basemodel,random_state = 42)
param_grids = {'base_estimator__alpha' : [1.0,0.001,0.1,0.01],'base_estimator__l1_ratio' : [0.1,0.4,0.7,0.9],'learning_rate' : [0.01,0.1,1.0,3.0],'n_estimators':[5,10,20,30,35],'loss':['linear', 'square', 'exponential']}
adaelastic_reg = GridSearchCV(estimator = adaelastic_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
adaelastic_reg.fit(X_train_poly,y_train)
adaelastic_reg.best_estimator_

Fitting 5 folds for each of 960 candidates, totalling 4800 fits
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END base_estimator__alpha=1.0, base_estimator__l1_ratio=0.1, learning_rate=0.01, loss=linear, n_estimators=10; total time=   0

AdaBoostRegressor(base_estimator=ElasticNet(alpha=0.1, l1_ratio=0.1,
                                            random_state=42),
                  learning_rate=0.1, loss='exponential', n_estimators=20,
                  random_state=42)

AdaBoostRegressor(base_estimator=ElasticNet(alpha=0.1, l1_ratio=0.1,
                                            random_state=42),
                  learning_rate=0.1, loss='exponential', n_estimators=20,
                  random_state=42)

In [48]:
adaelastic_reg = AdaBoostRegressor(base_estimator=ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42), learning_rate=0.1, loss='exponential', n_estimators=20, random_state=42)
adaelastic_reg.fit(X_train_poly,y_train)
y_pred = adaelastic_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'ADABOOST_elasticnet')
result_dataset

r2_score for ADABOOST_elasticnet :0.8144092945104184
adjusted_r2_score for ADABOOST_elasticnet :0.7869924857449121
median_absolute_error for ADABOOST_elasticnet :1.3639208090760917
mean_squared_error for ADABOOST_elasticnet :13.610068552944336
mean_absolute_percentage_error for ADABOOST_elasticnet :0.11442200941174359
mean_absolute_error for ADABOOST_elasticnet :2.1631789705315962


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.814409,0.0,0.0,0.0,0.0,0.0
adjusted_r2_score,0.894140,0.622384,0.678173,0.786992,0.0,0.0,0.0,0.0,0.0
median_absolute_error,1.192976,2.366688,2.134024,1.363921,0.0,0.0,0.0,0.0,0.0
mean_squared_error,6.763885,24.127699,20.563041,13.610069,0.0,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.114422,0.0,0.0,0.0,0.0,0.0
mean_absolute_error,1.827128,3.176261,3.070067,2.163179,0.0,0.0,0.0,0.0,0.0


# lets start with ADAboost with SVR

In [29]:
basemodel = SVR()
adasvr_reg = AdaBoostRegressor(base_estimator = basemodel,random_state = 42)
param_grids = {'base_estimator__C' : [1.0,0.001,0.1,0.01],'base_estimator__kernel' : ['poly','rbf'],'learning_rate' : [0.01,0.1,1.0],'n_estimators':[5,10,20,30,35]}
adasvr_reg = GridSearchCV(estimator = adasvr_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
adasvr_reg.fit(X_train,y_train)
adasvr_reg.best_estimator_

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=5; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=5; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=5; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=5; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=5; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=10; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=10; total time=   0.1s
[CV] END base_estimator__C=1.0, base_estimator__kernel=poly, learning_rate=0.01, n_estimators=10; total time=   0.1s
[CV] E

AdaBoostRegressor(base_estimator=SVR(), n_estimators=10, random_state=42)

daBoostRegressor(base_estimator=SVR(), n_estimators=10, random_state=42)

In [49]:
adasvr_reg = AdaBoostRegressor(base_estimator=SVR(), n_estimators=10, random_state=42)
adasvr_reg.fit(X_train,y_train)
y_pred = adasvr_reg.predict(X_test)
metrics(y_test,y_pred,model = 'ADABOOST_svr')
result_dataset

r2_score for ADABOOST_svr :0.7548249182950423
adjusted_r2_score for ADABOOST_svr :0.7186058721340827
median_absolute_error for ADABOOST_svr :1.6498110431421935
mean_squared_error for ADABOOST_svr :17.979616278065837
mean_absolute_percentage_error for ADABOOST_svr :0.15804055957167681
mean_absolute_error for ADABOOST_svr :2.7020008183326727


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.814409,0.754825,0.0,0.0,0.0,0.0
adjusted_r2_score,0.894140,0.622384,0.678173,0.786992,0.718606,0.0,0.0,0.0,0.0
median_absolute_error,1.192976,2.366688,2.134024,1.363921,1.649811,0.0,0.0,0.0,0.0
mean_squared_error,6.763885,24.127699,20.563041,13.610069,17.979616,0.0,0.0,0.0,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.114422,0.158041,0.0,0.0,0.0,0.0
mean_absolute_error,1.827128,3.176261,3.070067,2.163179,2.702001,0.0,0.0,0.0,0.0


# lets start with ADAboost with DT

In [31]:
basemodel = DecisionTreeRegressor(random_state = 42)
adadt_reg = AdaBoostRegressor(base_estimator = basemodel,random_state = 42)
param_grids = {'base_estimator__max_depth' : [1,2,3,4,5],'learning_rate' : [0.01,0.1,1.0,3.0],'n_estimators':[5,10,20,30,35],'loss':['linear', 'square', 'exponential']}
adadt_reg = GridSearchCV(estimator = adadt_reg,param_grid = param_grids,cv = 5, scoring = 'r2', verbose = 2)
adadt_reg.fit(X_train_poly,y_train)
adadt_reg.best_estimator_

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=5; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=10; total time=   0.0s
[CV] END base_estimator__max_depth=1, learning_rate=0.01, loss=linear, n_estimators=1

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5,
                                                       random_state=42),
                  loss='square', n_estimators=30, random_state=42)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5,
                                                       random_state=42),
                  loss='square', n_estimators=30, random_state=42)

In [50]:
adadt_reg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5,random_state=42),loss='square', n_estimators=30, random_state=42)
adadt_reg.fit(X_train_poly,y_train)
y_pred = adadt_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'ADABOOST_dt')
result_dataset

r2_score for ADABOOST_dt :0.8199076225697884
adjusted_r2_score for ADABOOST_dt :0.7933030668130526
median_absolute_error for ADABOOST_dt :1.5448088721094697
mean_squared_error for ADABOOST_dt :13.206855355294175
mean_absolute_percentage_error for ADABOOST_dt :0.12379868728050379
mean_absolute_error for ADABOOST_dt :2.284990762192272


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.814409,0.754825,0.819908,0.0,0.0,0.0
adjusted_r2_score,0.894140,0.622384,0.678173,0.786992,0.718606,0.793303,0.0,0.0,0.0
median_absolute_error,1.192976,2.366688,2.134024,1.363921,1.649811,1.544809,0.0,0.0,0.0
mean_squared_error,6.763885,24.127699,20.563041,13.610069,17.979616,13.206855,0.0,0.0,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.114422,0.158041,0.123799,0.0,0.0,0.0
mean_absolute_error,1.827128,3.176261,3.070067,2.163179,2.702001,2.284991,0.0,0.0,0.0


# lets implement XGBoost 

In [36]:
xgboost_reg = XGBRegressor(objective ='reg:squarederror',random_state = 42)
param_grids = {'max_depth' : [1,2,3,4],'learning_rate' : [0.1,1.0,0.01],'n_estimators':[36,34,35],'subsample': [0.8,0.6],'colsample_bytree': [0.3,0.4,0.5] ,'reg_alpha':[0.7,0.6],'reg_lambda':[0.1,0.0,0.2]}
xgboost_reg = GridSearchCV(estimator = xgboost_reg,param_grid = param_grids,cv = 5, scoring = 'accuracy', verbose = 2)
xgboost_reg.fit(X_train_poly,y_train)
xgboost_reg.best_estimator_

Streaming output truncated to the last 5000 lines.
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.6; total time=   0.0s
[CV] END colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimat

XGBRegressor(colsample_bytree=0.3, max_depth=1, n_estimators=36,
             objective='reg:squarederror', random_state=42, reg_alpha=0.7,
             reg_lambda=0.1, subsample=0.8)

XGBRegressor(colsample_bytree=0.3, max_depth=1, n_estimators=36,
             objective='reg:squarederror', random_state=42, reg_alpha=0.7,
             reg_lambda=0.1, subsample=0.8)

In [51]:
xgboost_reg = XGBRegressor(colsample_bytree=0.3, max_depth=1, n_estimators=36,objective='reg:squarederror', random_state=42, reg_alpha=0.7,reg_lambda=0.1, subsample=0.8)
xgboost_reg.fit(X_train_poly,y_train)
y_pred = xgboost_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'XGBOOST')
result_dataset

r2_score for XGBOOST :0.7238154924042721
adjusted_r2_score for XGBOOST :0.6830155083276305
median_absolute_error for XGBOOST :1.90279312133789
mean_squared_error for XGBOOST :20.253654792265678
mean_absolute_percentage_error for XGBOOST :0.15526437347244867
mean_absolute_error for XGBOOST :2.826629674200918


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.814409,0.754825,0.819908,0.723815,0.0,0.0
adjusted_r2_score,0.894140,0.622384,0.678173,0.786992,0.718606,0.793303,0.683016,0.0,0.0
median_absolute_error,1.192976,2.366688,2.134024,1.363921,1.649811,1.544809,1.902793,0.0,0.0
mean_squared_error,6.763885,24.127699,20.563041,13.610069,17.979616,13.206855,20.253655,0.0,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.114422,0.158041,0.123799,0.155264,0.0,0.0
mean_absolute_error,1.827128,3.176261,3.070067,2.163179,2.702001,2.284991,2.826630,0.0,0.0


# lets implement Lgboost

In [38]:
lgbm_reg = LGBMRegressor(random_state = 42)
param_grids = {'boosting_type':['gbdt','dart','goss','rf'],'max_depth' : [1,2,3,4],'learning_rate' : [0.1,1.0,0.01],'n_estimators':[36,34,35],'subsample': [0.8,0.6],'colsample_bytree': [0.3,0.4,0.5] ,'reg_alpha':[0.7,0.6],'reg_lambda':[0.1,0.0,0.2]}
lgbm_reg = GridSearchCV(estimator = lgbm_reg,param_grid = param_grids,cv = 5, scoring = 'accuracy', verbose = 2)
lgbm_reg.fit(X_train_poly,y_train)
lgbm_reg.best_estimator_

Streaming output truncated to the last 5000 lines.
[CV] END boosting_type=rf, colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END boosting_type=rf, colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END boosting_type=rf, colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END boosting_type=rf, colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END boosting_type=rf, colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0, subsample=0.8; total time=   0.0s
[CV] END boosting_type=rf, colsample_bytree=0.3, learning_rate=0.01, max_depth=1, n_estimators=36, reg_alpha=0.6, reg_lambda=0.0,

LGBMRegressor(colsample_bytree=0.3, max_depth=1, n_estimators=36,
              random_state=42, reg_alpha=0.7, reg_lambda=0.1, subsample=0.8)

LGBMRegressor(colsample_bytree=0.3, max_depth=1, n_estimators=36,
              random_state=42, reg_alpha=0.7, reg_lambda=0.1, subsample=0.8)

In [52]:
lgbm_reg = LGBMRegressor(colsample_bytree=0.3, max_depth=1, n_estimators=36,random_state=42, reg_alpha=0.7, reg_lambda=0.1, subsample=0.8)
lgbm_reg.fit(X_train_poly,y_train)
y_pred = lgbm_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'LGBM')
result_dataset

r2_score for LGBM :0.7503079297119424
adjusted_r2_score for LGBM :0.7134216011466612
median_absolute_error for LGBM :2.079079871671225
mean_squared_error for LGBM :18.310864139356536
mean_absolute_percentage_error for LGBM :0.1570927966279661
mean_absolute_error for LGBM :2.709816111270809


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.814409,0.754825,0.819908,0.723815,0.750308,0.0
adjusted_r2_score,0.894140,0.622384,0.678173,0.786992,0.718606,0.793303,0.683016,0.713422,0.0
median_absolute_error,1.192976,2.366688,2.134024,1.363921,1.649811,1.544809,1.902793,2.079080,0.0
mean_squared_error,6.763885,24.127699,20.563041,13.610069,17.979616,13.206855,20.253655,18.310864,0.0
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.114422,0.158041,0.123799,0.155264,0.157093,0.0
mean_absolute_error,1.827128,3.176261,3.070067,2.163179,2.702001,2.284991,2.826630,2.709816,0.0


# lets implement CATBOOST

In [40]:
catboost_reg = CatBoostRegressor(random_state = 42)
param_grids = {'max_depth' : [1,2,3,4],'learning_rate' : [0.1,1.0,0.01],'loss_function': ['RMSE'] , 'n_estimators':[36,34,35],'subsample': [0.8,0.6],'colsample_bylevel': [0.3,0.4,0.5] ,'reg_lambda':[0.1,0.0,0.2]}
catboost_reg = GridSearchCV(estimator = catboost_reg,param_grid = param_grids,cv = 5, scoring = 'accuracy', verbose = 2)
catboost_reg.fit(X_train_poly,y_train)
catboost_reg.best_estimator_

Streaming output truncated to the last 5000 lines.
21:	learn: 8.1072766	total: 64ms	remaining: 34.9ms
22:	learn: 8.0661243	total: 68.5ms	remaining: 32.7ms
23:	learn: 8.0218342	total: 72.2ms	remaining: 30.1ms
24:	learn: 7.9783659	total: 74.6ms	remaining: 26.8ms
25:	learn: 7.9309532	total: 77.4ms	remaining: 23.8ms
26:	learn: 7.8779770	total: 80.5ms	remaining: 20.9ms
27:	learn: 7.8321453	total: 83.2ms	remaining: 17.8ms
28:	learn: 7.7825882	total: 86.2ms	remaining: 14.9ms
29:	learn: 7.7344650	total: 89.3ms	remaining: 11.9ms
30:	learn: 7.6897809	total: 92.3ms	remaining: 8.93ms
31:	learn: 7.6488498	total: 95ms	remaining: 5.94ms
32:	learn: 7.6030592	total: 98.7ms	remaining: 2.99ms
33:	learn: 7.5540719	total: 101ms	remaining: 0us
[CV] END colsample_bylevel=0.5, learning_rate=0.01, loss_function=RMSE, max_depth=3, n_estimators=34, reg_lambda=0.0, subsample=0.8; total time=   0.2s
0:	learn: 9.1118030	total: 2.58ms	remaining: 85.3ms
1:	learn: 9.0528743	total: 5.22ms	remaining: 83.5ms
2:	learn: 9.

In [41]:
catboost_reg.best_params_

{'colsample_bylevel': 0.3,
 'learning_rate': 0.1,
 'loss_function': 'RMSE',
 'max_depth': 1,
 'n_estimators': 36,
 'reg_lambda': 0.1,
 'subsample': 0.8}

In [53]:
catboost_reg = CatBoostRegressor(colsample_bylevel = 0.3,learning_rate = 0.1,loss_function = 'RMSE',max_depth = 1,n_estimators = 36,reg_lambda = 0.1,subsample = 0.8,random_state = 42,verbose = 0)
catboost_reg.fit(X_train_poly,y_train)
y_pred = catboost_reg.predict(X_test_poly)
metrics(y_test,y_pred,model = 'CATBOOST')
result_dataset

r2_score for CATBOOST :0.6711960276334714
adjusted_r2_score for CATBOOST :0.6226227135338706
median_absolute_error for CATBOOST :1.908307655739133
mean_squared_error for CATBOOST :24.112439211779826
mean_absolute_percentage_error for CATBOOST :0.16927632323921507
mean_absolute_error for CATBOOST :3.002155360687549


,GBDT,ADABOOST_linear,ADABOOST_polynomial,ADABOOST_elasticnet,ADABOOST_svr,ADABOOST_dt,XGBOOST,LGBM,CATBOOST
r2_score,0.907766,0.670988,0.719597,0.814409,0.754825,0.819908,0.723815,0.750308,0.671196
adjusted_r2_score,0.894140,0.622384,0.678173,0.786992,0.718606,0.793303,0.683016,0.713422,0.622623
median_absolute_error,1.192976,2.366688,2.134024,1.363921,1.649811,1.544809,1.902793,2.079080,1.908308
mean_squared_error,6.763885,24.127699,20.563041,13.610069,17.979616,13.206855,20.253655,18.310864,24.112439
mean_absolute_percentage_error,0.096589,0.168010,0.172633,0.114422,0.158041,0.123799,0.155264,0.157093,0.169276
mean_absolute_error,1.827128,3.176261,3.070067,2.163179,2.702001,2.284991,2.826630,2.709816,3.002155
